## Scrape media impact funders

Source: maps.foundationcenter.org

### Try scrape with requests and beautiful soup

Of course it didn't work :)

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

response = requests.get('https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-139.219,-31.354,135,66.513&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=2')

response.status_code

Tried accessing the api, but do not have authentication

In [ ]:
headers = { 
   'authority': '0nn8jwscqe.execute-api.us-east-1.amazonaws.com',
   'method': 'GET', 
   'path': '/prod/fm-elastic-prod-cds?queryType=list&view=fm-view-list&subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-139.219,-31.354,135,66.513&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=gm&windRoseAnd=undefined&zoom=2&listType=gm&sortColumn=amount&sortOrder=desc',
   'scheme': 'https',
  'accept': 'application/json, text/javascript, */*; q=0.01',
  'accept-encoding': 'gzip, deflate, br, zstd',
  'accept-language': 'en-US,en;q=0.9',
  'origin': 'https://maps.foundationcenter.org',
  'authentication': '',
  'content-type': 'application/json; charset=utf-8',
  'cookie': 'rdpirm01=',
  'dnt': '0',
  'referer': 'https://maps.foundationcenter.org/',
  'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': "macOS",
  'sec-fetch-dest': 'empty',                        
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
}

response = requests.get('https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-133.9453125,14.689881366618762,-37.6171875,55.825973254619015&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=4')
response.status_code

  # Parse the content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract data based on the provided selectors
grants_data = []

# Locate all rows of grant data (assuming each row has the required fields)
rows = soup.select("table tbody tr")  # Adjust if necessary for table structure
for row in rows:
    funder_name = row.select_one("a.gm-link").get_text(strip=True) if row.select_one("a.gm-link") else None
    recipient = row.select_one("a.recip-link").get_text(strip=True) if row.select_one("a.recip-link") else None
    year = row.select_one("td.cell-year").get_text(strip=True) if row.select_one("td.cell-year") else None
    value = row.select_one("td.sorting_2").get_text(strip=True) if row.select_one("td.sorting_2") else None

    # Append to the list if data is found
    grants_data.append({
        "funder_name": funder_name,
        "recipient": recipient,
        "year": year,
        "value": value
    })

### Try Playwright

In [52]:
import asyncio
from playwright.async_api import async_playwright

async def download_csv():
    async with async_playwright() as p:
        # Launch the browser (set headless=False if you want to see the browser actions)
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        
        # Specify the download path for Playwright to store files
        # context.set_default_download_path("/Users/yanans/Desktop/donor/")  # Set your download path

        # Open a new page
        page = await context.new_page()

        # Go to the target URL
        await page.goto("https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-133.9453125,14.689881366618762,-37.6171875,55.825973254619015&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=4")

        # Loop through the dynamic views
        for i in range(1, 6):  # Assume 5 dynamic views/pages to navigate through
            # Perform actions to navigate to the next view (e.g., clicking a "Next" button)
            # Adjust selector to match the button for switching pages
            # await page.click("button#next-page")

            # await page.get_by_text(f'{i}').click()

            # pagi = page.locator('.pagination')
            # await locator.get_by_text(f'{i}').click()
            next_page = i + 1 
            # await page.click("ul.pagination li a:has-text('1')")


            # Wait for the CSV download button to be ready
            # locator = page.locator(".export-csv")
            # locator = page.locator(".widget-export-tools > span:nth-child(2)").first()
            # locator = page.locator("#profile-modal #fm-pdf-csv-2 span").nth(1)
            # 2) <span class="spriteimg export-csv"></span> aka locator("#modal-all-list #fm-pdf-csv-2 span").nth(1)
            # 3) <span class="spriteimg export-csv"></span> aka locator("#profile-modal #fm-pdf-csv-2 span").nth(1)
            # 4) <span class="spriteimg export-csv"></span> aka locator("#profile-modal-dashboard #fm-pdf-csv-2 span").nth(1)
            # 5) <span class="spriteimg export-csv"></span> aka locator("#regional-modal-dashboard #fm-pdf-csv-2 span").nth(1)
            # locator.wait_for(state="visible")
            # await locator.click()
            # await page.wait_for_selector("span#spriteimg export-csv")  # Adjust selector as needed

            # grant = page.locator("li.grants-link a#list-screen-all")
            # if await page.locator("li.grants-link a#list-screen-all"):
            await page.click("li.grants-link")


            # Trigger download and wait for it to complete
            # download = await page.wait_for_event("download", lambda download: download.url.endswith(".csv"))
            # await download.save_as("/Users/yanans/Desktop/donor/" + f"data_view_{i}.csv")

            print(f"Downloaded CSV for view {i}")
            break

        # Close the browser
        await browser.close()

# Run the async function
# asyncio.run(download_csv())
# RuntimeError: asyncio.run() cannot be called from a running event loop
await download_csv()


TimeoutError: Page.click: Timeout 30000ms exceeded.
Call log:
waiting for locator("li.grants-link")
  -   locator resolved to 2 elements. Proceeding with the first one: <li class="grants-link">…</li>
  - attempting click action
  -   waiting for element to be visible, enabled and stable
  -   element is not visible
  - retrying click action, attempt #1
  -   waiting for element to be visible, enabled and stable
  -   element is not stable
  - retrying click action, attempt #2
  -   waiting 20ms
  -   waiting for element to be visible, enabled and stable
  -   element is visible, enabled and stable
  -   scrolling into view if needed
  -   done scrolling
  -   <li>…</li> from <div id="welcome-modal" aria-hidden="false" class="modal fade in">…</div> subtree intercepts pointer events
  - retrying click action, attempt #3
  -   waiting 100ms
  -   waiting for element to be visible, enabled and stable
  -   element is visible, enabled and stable
  -   scrolling into view if needed
  -   don

### Try Selenium

Automate their csv download by clicking on their "Download this data" button

But they only provide the first 100 rows no matter what page it is on

In [9]:
# Import Module
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

 
# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
# driver = webdriver.Chrome()
 
# Open URL
driver.get(
    'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=all&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=all&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=8')
 
# Start Searching
try:
    driver.find_element(By.ID, "fm-start-searching").click()
    # driver.find_element(By.CLASS_NAME, "btn btn-primary").click()
except Exception as e:
    print(e)
else:
    # driver.find_element(By.CLASS_NAME, "btn btn-primary")
    i = 1
    while i<3:
        # Click "Grants" button
        driver.find_element(By.CLASS_NAME, 'grants-link').click()
    
        # Click "Download this data" button
        label = driver.find_element(By.CLASS_NAME, "widget-export-tools")
        try:
            # label.find_element(By.CLASS_NAME, 'spriteimg.export-csv')
            label.find_element(By.CLASS_NAME, 'spriteimg.export-csv').click()
        except Exception as e:
            print(e)
        else:
            print(f"Button clicked for page{i+1}")
            page = driver.find_element(By.CLASS_NAME, 'table-pagination').find_element(By.CLASS_NAME, 'pagination').find_element(By.CLASS_NAME, 'active')
            next_page = await page.find_element(By.XPATH,"following-sibling::li").click()
            i+=1
            # driver.find_element(By.XPATH, '//*[@id="fm-list-1grantfm-list-table_wrapper"]/div[2]/div[2]/div/ul/li[2]/following-sibling::li').find_element(By.XPATH, '/a').click()

driver.quit()
# Click on Download Button
# driver.find_element_by_id('link-to-download').click()

Button clicked for page2


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=129.0.6668.101)
Stacktrace:
0   chromedriver                        0x0000000101294500 cxxbridge1$str$ptr + 1917112
1   chromedriver                        0x000000010128c890 cxxbridge1$str$ptr + 1885256
2   chromedriver                        0x0000000100e9c3a8 cxxbridge1$string$len + 89024
3   chromedriver                        0x0000000100ee1838 cxxbridge1$string$len + 372816
4   chromedriver                        0x0000000100ed727c cxxbridge1$string$len + 330388
5   chromedriver                        0x0000000100ed6d18 cxxbridge1$string$len + 329008
6   chromedriver                        0x0000000100f1ab7c cxxbridge1$string$len + 607124
7   chromedriver                        0x0000000100ed5374 cxxbridge1$string$len + 322444
8   chromedriver                        0x0000000100ed5fc4 cxxbridge1$string$len + 325596
9   chromedriver                        0x000000010125bd2c cxxbridge1$str$pt

#### Scrape grants details

Scrape a single page

In [ ]:
# Scrape a single page

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Store data
funder, funder_addr, recipient, recipient_addr, amount, year, area, duration, title, subject, population, strategy, transaction, desc, source = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
# Open browser
url = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2024&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=8'

# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get(url)

# Start Searching
try:
    driver.find_element(By.ID, "fm-start-searching").click()
    # driver.find_element(By.CLASS_NAME, "btn btn-primary").click()
except Exception as e:
    print(e)
else:
    # storing the current window handle to get back to dashboard
    main_page = driver.current_window_handle

    # Click "Grants"
    grants = driver.find_element(By.CLASS_NAME, 'grants-link')
    grants.click()
    
    time.sleep(2)
    # Locate a row
    all_list = driver.find_element(By.ID, 'list-screen-all')    #<div class="tab-pane fade active in" id="list-screen-all">
    tbody = all_list.find_element(By.TAG_NAME, 'tbody')         # <tbody>
    row = tbody.find_element(By.CLASS_NAME, 'gradeX')

    
    order = 0
    # Each page has 100 rows
    for i in range(100):
        # time.sleep(2)

        # Press button
        try:
            WebDriverWait(row, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "table-detail-icon"))).click()    # Wait until active
        except:
            time.sleep(5)
            WebDriverWait(row, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "table-detail-icon"))).click()    # Wait until active

        # Wait for the window to pop up
        wait = WebDriverWait(driver, timeout=2)
        window = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="fm-list-1popupprofile-modal-new"]')))

        # Collect details
        amount.append(window.find_element(By.CLASS_NAME, "modal-grant-details").find_element(By.ID, "grant-detail-total-amount").text)
        funder.append(window.find_element(By.ID, "grant-detail-grantmaker").text)
        funder_addr.append(window.find_element(By.ID, "grant-detail-gm-address").text)
        recipient.append(window.find_element(By.ID, "grant-detail-recipient").text)
        recipient_addr.append(window.find_element(By.ID, "grant-detail-recipient-address").text)
        year.append(window.find_element(By.ID, "grant-detail-year").text)
        # print(year)
        area.append(window.find_element(By.ID, 'grant-detail-program-area').text)
        # print(area)
        duration.append(window.find_element(By.ID, 'grant-detail-duration').text)
        # print(duration)
        title.append(window.find_element(By.ID, 'grant-detail-title').text)
        # print(title)
        subject.append(window.find_element(By.ID, 'grant-detail-subject').text)
        # print(subjects)
        population.append(window.find_element(By.ID, 'grant-detail-population').text)
        # print(population)
        strategy.append(window.find_element(By.ID, 'grant-detail-strategy-type').text)
        # print(strategy)
        transaction.append(window.find_element(By.ID, 'grant-detail-transaction-type').text)
        # print(type)        
        desc.append(window.find_element(By.ID, 'grant-detail-description').text)
        source.append(window.find_element(By.ID, 'data-license').find_element(By.TAG_NAME, "a").get_attribute('href'))
        
        window.find_element(By.CLASS_NAME, 'close').click()

        # Find next sibling row until done
        time.sleep(2)
        try:
            row = row.find_element(By.XPATH,"following-sibling::tr")
        except Exception as e:
            print(e)
        break
            

driver.quit()



Scrape multiple pages

In [ ]:
# Scrape multiple pages
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Store data
funder, funder_addr, recipient, recipient_addr, amount, year, area, duration, title, subject, population, strategy, transaction, desc, source = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[]

# Get urls
# url_2024 = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2024&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=8'
# url_2023 = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2023&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=0&maxGrantAmt=0&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=gm&windRoseAnd=undefined&zoom=8'
# url_2023_1 = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2023&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=&maxGrantAmt=120&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=8'
# url_2022 = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2022&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=&maxGrantAmt=&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=grant&windRoseAnd=undefined&zoom=8'
# url_2022_1 = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2022&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=&maxGrantAmt=124999&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=gm&windRoseAnd=undefined&zoom=8'
# url_2022_2 = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2022&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=7500&maxGrantAmt=7501&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=gm&windRoseAnd=undefined&zoom=8'
url_2021 = 'https://maps.foundationcenter.org/#/list/?subjects=M5000&popgroups=all&years=2021&location=6252001&excludeLocation=0&geoScale=ADM0&layer=gm&boundingBox=-103.5736083984375,32.189559980413584,-96.416015625,34.93548199355901&gmOrgs=all&recipOrgs=all&tags=all&keywords=&pathwaysOrg=&pathwaysType=&acct=media&typesOfSupport=all&transactionTypes=TA,TD,TB,TC,TG&amtRanges=all&minGrantAmt=&maxGrantAmt=&gmTypes=all&recipTypes=all&minAssetsAmt=0&maxAssetsAmt=0&minGivingAmt=0&maxGivingAmt=0&andOr=0&includeGov=1&custom=all&customArea=all&indicator=&dataSource=oecd&chartType=trends&multiSubject=1&listType=gm&windRoseAnd=undefined&zoom=8'



# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get(url_2021)

# Start Searching
try:
    driver.find_element(By.ID, "fm-start-searching").click()
except Exception as e:
    print(e)
else:
    # storing the current window handle to get back to dashboard
    main_page = driver.current_window_handle

    # Click "Grants"
    grants = driver.find_element(By.CLASS_NAME, 'grants-link')
    grants.click()
    
    time.sleep(2)
    
    # Start from page 1
    num = 1

    # Change this number to the total page number
    while num <=1:
        
        # # Locate a row
        all_list = driver.find_element(By.ID, 'list-screen-all')    #<div class="tab-pane fade active in" id="list-screen-all">
        tbody = all_list.find_element(By.TAG_NAME, 'tbody')         # <tbody>
        row = tbody.find_element(By.CLASS_NAME, 'gradeX')
        
        # Each page has 100 rows
        for i in range(100):
            
            # Press button
            try:
                WebDriverWait(row, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "table-detail-icon"))).click()    # Wait until active
            except:
                time.sleep(5)
                WebDriverWait(row, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "table-detail-icon"))).click()    # Wait until active

            # row.find_element(By.CLASS_NAME, "table-detail-icon").click()

            # Wait for the window to pop up
            wait = WebDriverWait(driver, timeout=2)
            window = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="fm-list-1popupprofile-modal-new"]')))

            # Collect details
            amount.append(window.find_element(By.CLASS_NAME, "modal-grant-details").find_element(By.ID, "grant-detail-total-amount").text)
            funder.append(window.find_element(By.ID, "grant-detail-grantmaker").text)
            funder_addr.append(window.find_element(By.ID, "grant-detail-gm-address").text)
            recipient.append(window.find_element(By.ID, "grant-detail-recipient").text)
            recipient_addr.append(window.find_element(By.ID, "grant-detail-recipient-address").text)
            year.append(window.find_element(By.ID, "grant-detail-year").text)
            area.append(window.find_element(By.ID, 'grant-detail-program-area').text)
            duration.append(window.find_element(By.ID, 'grant-detail-duration').text)
            title.append(window.find_element(By.ID, 'grant-detail-title').text)
            subject.append(window.find_element(By.ID, 'grant-detail-subject').text)
            population.append(window.find_element(By.ID, 'grant-detail-population').text)
            strategy.append(window.find_element(By.ID, 'grant-detail-strategy-type').text)
            transaction.append(window.find_element(By.ID, 'grant-detail-transaction-type').text)
            desc.append(window.find_element(By.ID, 'grant-detail-description').text)
            source.append(window.find_element(By.ID, 'data-license').find_element(By.TAG_NAME, "a").get_attribute('href'))
            
            # Close the detail page
            window.find_element(By.CLASS_NAME, 'close').click()

            # Find next sibling row until done
            time.sleep(1)
            try:
                row = row.find_element(By.XPATH,"following-sibling::tr")
            except Exception as e:
                print(f"Finished page {num}")
                break

        # Change this number to the total page number
        if num == 1:
            break
        else:
            # Find the active page
            page = all_list.find_element(By.CLASS_NAME, 'table-pagination').find_element(By.CLASS_NAME, 'pagination').find_element(By.CLASS_NAME, "active").find_element(By.XPATH,"following-sibling::li")
            
            # Click the next page
            next_page = WebDriverWait(page, 10).until(EC.element_to_be_clickable((By.TAG_NAME,"a"))).click()

            num+=1
            time.sleep(1)

            

driver.quit()



Finished page 1


In [ ]:
import pandas as pd

funder = {"amount":amount,
            "year": year, 
            "area": area, 
            "duration": duration, 
            "title": title, 
            "subject": subject,
            "population": population, 
            "strategy": strategy, 
            "transaction": transaction,
            "funder": funder,
            "funder_addr": funder_addr, 
            "recipient": recipient, 
            "recipient_addr": recipient_addr,
            "description": desc, 
            "source": source
            
}

funder_df_7500_last = pd.DataFrame(data = funder)
funder_df_7500_last.tail()

,amount,year,area,duration,title,subject,population,strategy,transaction,funder,funder_addr,recipient,recipient_addr,description,source
45,"TOTAL AMOUNT: $7,500",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,"SUBJECT:Journalism, news and information; Jour...",POPULATION SERVED: Not Specified,SUPPORT STRATEGY: Not Specified,TRANSACTION TYPE: Cash grants,The Strauss Family Foundation,"Greenwood Village, CO United States",The Current Media Inc,"Lafayette, LA United States",CHARITABLE PURPOSE,http://foundationcenter.org/gain-knowledge/fou...
46,"TOTAL AMOUNT: $7,500",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,"SUBJECT:Journalism, news and information; Jour...",POPULATION SERVED: Academics,SUPPORT STRATEGY: Program support,TRANSACTION TYPE: Cash grants,The Thomas B Stanley Jr and Ruth B Stanley Fam...,"Orlando, FL United States",Kennesaw State University Foundation Inc,"Kennesaw, GA United States",CENTER FOR SUSTAINABLE JOURNALISM,http://foundationcenter.org/gain-knowledge/fou...
47,"TOTAL AMOUNT: $7,500",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,"SUBJECT:Journalism, news and information; Jour...",POPULATION SERVED: Academics,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Top Jewish Foundation Inc,"Tampa, FL United States",The Poynter Institute for Media Studies Inc,"St Petersburg, FL United States",GENERAL SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
48,"TOTAL AMOUNT: $7,500",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,"SUBJECT:Journalism, news and information; Jour...",POPULATION SERVED: Children and youth; Student...,SUPPORT STRATEGY: Advocacy; Outreach,TRANSACTION TYPE: Cash grants,Vegfund Inc,"Washington, DC United States",EDUCATED CHOICES PROGRAM,"Lawrenceville, GA United States",SUPPORT VEGAN ADVOCACY AND ENCOURAGE VEGAN LIF...,http://foundationcenter.org/gain-knowledge/fou...
49,"TOTAL AMOUNT: $7,500",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,"SUBJECT:Journalism, news and information; Jour...",POPULATION SERVED: Adults,SUPPORT STRATEGY: Advocacy; Outreach,TRANSACTION TYPE: Cash grants,Vegfund Inc,"Washington, DC United States",Friends of the Earth,"Washington, DC United States",SUPPORT VEGAN ADVOCACY AND ENCOURAGE VEGAN LIF...,http://foundationcenter.org/gain-knowledge/fou...


In [ ]:
# For year 2022, i had to split them into three groups
# Because there are more than 5,300 rows

# 125000 - max
funder_df_125000_last_1 = funder_df_125000_last.iloc[-10:]  # get the last ten
funder_125000 = pd.concat([funder_df, funder_df_125000_last_1], ignore_index=True)

# 7500 - 124999
# Get rid of 7500 
funder_df_no_7500 = funder_df_7500[funder_df_7500.amount != "TOTAL AMOUNT:  $7,500"]
funder_df_no_7500

# Add 7500
funder_7500 = pd.concat([funder_df_no_7500, funder_df_7500_last], ignore_index=True)

# min - 7499
funder_df_last_2 = funder_df_last.iloc[:-138]

# Merge and store locally
funder_2022 = pd.concat([funder_125000, funder_7500, funder_df_last_2], ignore_index=True)
# funder_2022.tail()
# funder_2022.to_csv("grants_2022.csv", index=False)

,amount,year,area,duration,title,subject,population,strategy,transaction,funder,funder_addr,recipient,recipient_addr,description,source
1000,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1001,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1002,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1003,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1004,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1005,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1006,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1007,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1008,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http://foundationcenter.org/gain-knowledge/fou...
1009,"TOTAL AMOUNT: $125,000",FISCAL YEAR: 2022,,GRANT DURATION: 1 year,,SUBJECT:Media access and policy; Freedom of Ex...,POPULATION SERVED: Not Specified,SUPPORT STRATEGY: General support,TRANSACTION TYPE: Cash grants,Wellspring Philanthropic Fund,"New York City, NY United States",Freedom Voices Network,"Paris, France",OPERATING SUPPORT,http

#### Clean Data

In [ ]:
def clean_df(funder_df_1):
    # Drop duplicates of the last row
    funder_df_1.tail()
    last_row = funder_df_1.iloc[-1]
    indices_to_remove = funder_df_1[(funder_df_1 == last_row).all(axis=1)].index[:-1]
    funder_df_1 = funder_df_1.drop(indices_to_remove)

    # Clean text
    funder_df_1.amount = funder_df_1.amount.str.replace('TOTAL AMOUNT:', '')
    funder_df_1.year = funder_df_1.year.str.replace('FISCAL YEAR:', '')
    funder_df_1.area = funder_df_1.area.str.replace('PROGRAM AREA:', '')
    funder_df_1.duration = funder_df_1.duration.str.replace('GRANT DURATION:', '')
    funder_df_1.title = funder_df_1.title.str.replace('GRANT TITLE:', '')
    funder_df_1.subject = funder_df_1.subject.str.replace('SUBJECT:', '')
    funder_df_1.population = funder_df_1.population.str.replace('POPULATION SERVED: ', '')
    funder_df_1.strategy = funder_df_1.strategy.str.replace('SUPPORT STRATEGY:', '')
    funder_df_1.transaction = funder_df_1.transaction.str.replace('TRANSACTION TYPE: ', '')

    # Remove space
    funder_df_1 = funder_df_1.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    
    # Reset index
    funder_df_1 = funder_df_1.reset_index()
    return funder_df_1

/var/folders/cn/fww9r0gd1mg20td4q8v12_lsknvlj7/T/ipykernel_10893/398715982.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  funder_df_1 = funder_df_1.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# Clean 2023
df_2023_clean = clean_df(df_2023_1)
df_2023_clean.tail()
df_2023_clean.to_csv('grants_2023_clean.csv', index=False)

# Clean 2022
df_2022_clean = clean_df(funder_2022)
df_2022_clean.to_csv('grants_2022_clean.csv', index=False)

/var/folders/cn/fww9r0gd1mg20td4q8v12_lsknvlj7/T/ipykernel_10893/398715982.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  funder_df_1 = funder_df_1.applymap(lambda x: x.strip() if isinstance(x, str) else x)
